In [31]:
import pandas as pd
import numpy as np
import newsdataapi
import json
import matplotlib.pyplot as plt
import re
import os
from dotenv import load_dotenv
from openai import OpenAI
import requests
from io import StringIO
import bcchapi
import openpyxl
from IPython.display import Markdown, display

load_dotenv()
client = OpenAI(api_key = os.getenv('OPEN_API_KEY'))

# MEXICO

In [8]:
token_mex = os.getenv('API_MEX')
def get_df_from_inegi(indicator_id, token = token_mex):
    url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicator_id}/es/0700/false/BIE/2.0/{token}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data["Series"][0]['OBSERVATIONS'])[["TIME_PERIOD","OBS_VALUE"]]
    return df

In [9]:
#indicador_id = '910406' #inflation

# First request for GDP proxy (737220)
df_gdp_proxy = get_df_from_inegi("737220")

# Second request for unemployment (444884)
df_unemployment = get_df_from_inegi("444884")

# Inflation
df_inflation_mex = pd.read_excel('Data/df_infl_mex.xlsx', skiprows=4)
df_inflation_mex = df_inflation_mex.rename(columns={'Periodos': 'Date', 'Precios e Inflación': 'Inflation'})
df_inflation_mex.columns = ['date', 'Inflation']

# Merge the dataframes on the date column
df_gdp_proxy.columns = ['date', 'IGAE']
df_unemployment.columns = ['date', 'unemployment']
df_mexico = pd.merge(df_gdp_proxy, df_unemployment, on='date', how='outer')

# Final merge
df_mexico = df_mexico.sort_values('date', ascending=False)
df_mexico = pd.merge(df_mexico, df_inflation_mex, on='date', how='outer')


# CHILE

In [4]:
api_chile = bcchapi.Siete(os.getenv('CORREO_CHILE'), os.getenv('PASSWORD_CHILE'))

df_chile = api_chile.cuadro(
    series = ["F032.IMC.IND.Z.Z.EP18.Z.Z.0.M", "F074.IPC.V12.Z.EP23.C.M", "F049.DES.TAS.INE9.10.M"],
    nombres = ["imacec", "ipc", "unemployment"],
    desde = "2024-01-01",
    hasta = "2025-03-31",
    frecuencia = "ME",
    observado = {"imacec":"last", "ipc":"last", "unemployment":"last"}
)   

# COSTA RICA

In [ ]:
token_cr = os.getenv('API_CR')
def get_df_from_cr(indicator_id, start_period, end_period, sub_level, indicator_name,
                   name = 'Ahias', token = token_cr, email = os.getenv('CORREO_CHILE')):
    
    url = 'https://gee.bccr.fi.cr/Indicadores/Suscripciones/WS/wsindicadoreseconomicos.asmx/ObtenerIndicadoresEconomicosXML'
    url = url + f'?Indicador={indicator_id}&FechaInicio={start_period}&FechaFinal={end_period}&Nombre={name}&SubNiveles={sub_level}'
    url = url + f'&CorreoElectronico={email}&Token={os.getenv("API_CR")}'
    
    response = requests.get(url)
    
    clean_text = response.text.replace('<?xml version="1.0" encoding="utf-8"?>', '')
    clean_text = re.sub(r'<[^>]+>', '', clean_text)
    clean_text = re.sub(r'(&lt;)|(&gt)', '', clean_text)
    clean_text = re.sub(r"/[A-Z]+_[A-Z]+", '', clean_text)
    clean_text = re.sub(r"(Datos_de_INGC011_CAT_INDICADORECONOMIC;)|(INGC011_CAT_INDICADORECONOMIC;)", '', clean_text)
    clean_text = re.sub(r'/', '', clean_text)
    clean_text = ([line.strip() for line in clean_text.split('\n') if line.strip()])
    
    dates = np.array([])
    values = np.array([])

    for element in clean_text:
        divided = re.split(r";", element)
        if divided[0] == 'DES_FECHA':
            date_str = divided[1].split('T')[0]
            dates = np.append(dates, pd.to_datetime(date_str))
        if divided[0] == 'NUM_VALOR':      
            values = np.append(values, float(divided[1]))
            
    df = pd.DataFrame({'date': dates,
                       indicator_name: values})
    return df

In [10]:
df_cr_gfp_proxy = get_df_from_cr(87764, '01/01/2024', '18/04/2025', 'N', 'IMAE') #gdp proxy
df_cr_unempl = get_df_from_cr(23633, '01/01/2024', '18/04/2025', 'N', 'unemloyment') #unemloyment
df_cr_inf = get_df_from_cr(89638, '01/01/2024', '18/04/2025', 'N', 'var_ipc') #inflation

df_cr = pd.merge(df_cr_gfp_proxy, df_cr_unempl, on='date', how='outer')
df_cr = pd.merge(df_cr, df_cr_inf, on = 'date', how = 'outer')


# LLM's

Creating a personalize text based upon data retreived from the API's of these countries.

In [ ]:
# Convert dataframes to CSV format in memory
chile_csv = df_chile.to_csv(index=False)
cr_csv = df_cr.to_csv(index=False)
mexico_csv = df_mexico.to_csv(index=False)

# Combine the data into a single message for the API
data_message = f"""
Data for Chile:
{chile_csv}

Data for Costa Rica:
{cr_csv}

Data for Mexico:
{mexico_csv}
"""

In [45]:
prompt = f"""Your task is to analyze each of the following datasets, which contain the following indicators: inflation, unemployment, and short-term economic activity for three countries—Mexico, Chile, and Costa Rica.
Write a short, 3-line paragraph summarizing the recent economic situation of each country, suitable for inclusion in a dashboard or executive summary.
Each dataset includes the following columns:
    - Date: Monthly date. This could be a regular date corresponding to the end of the month, or a year/month format (e.g., 2025/05, where 2025 is the year and 05 is the month).
    - Inflation: Monthly year-on-year inflation rate. Possible column names include inflation or any that contain "IPC" (in upper or lower case).
    - Unemployment: Monthly unemployment rate. Possible column name: unemployment.
    - Short-term Economic Activity Indicator: A statistical index. Possible column names: IMAE, IMACE or IGAE.
Instructions:
- For the short-term economic activity indicator, calculate the year-on-year percentage change before writing your overview.
- Include an analysis comparing the performance so far this year to the same period last year. (This is a year-to-date comparison: e.g., Jan–Apr 2025 vs. Jan–Apr 2024)
- Include an analysis of the most recent available month compared to the immediately preceding month. (This is a month-over-month comparison: e.g., April 2025 vs. March 2025)
- Be informative.
- Indicate which date(s) from the dataset your analysis is based on, so it's clear what time period is being referenced.
Output:
For each country, provide two versions of the brief:
    -One for a specialized audience (e.g., other economists).
    -One for the general public (with little or no knowledge of economics).
Begin each output with the country name. Label each version clearly as ‘Expert’ or ‘General Public.’ Each version must be a maximum of 3 lines. You can follow the next example:    **Country**
        *Expert*
            Here is the output for the specialized audience.
        *General Public*
            Here id the output for the general public.
Here is the data to analyze:
{data_message}
"""

In [46]:
response = client.responses.create(
    model = "gpt-4o-mini",
    input = [
        {
            "role": "developer",
            "content": "You are a prominent economist and data analyst tasked with delivering key insights into macroeconomic trends"
        },
        {
            "role": "user",
            "content": prompt
        }],
    temperature = 0.5    
)

In [47]:
display(Markdown(response.output_text))

**Chile**  
*Expert*  
Chile's IMAEC shows a year-on-year increase of approximately 7.6% for January-April 2025 compared to the same period in 2024, while unemployment has decreased slightly to 8.3%. Month-over-month, inflation has risen to 4.9% in February 2025, indicating potential inflationary pressures.  
*General Public*  
Chile's economy is growing, with a 7.6% increase in economic activity compared to last year, and unemployment has dropped a bit to 8.3%. However, inflation is rising, reaching 4.9% in February 2025, which might affect prices.

---

**Costa Rica**  
*Expert*  
Costa Rica's IMAE shows a year-on-year increase of 4.4% for January-April 2025 compared to the same period in 2024, while the unemployment rate has improved to 9.7%. Month-over-month, inflation has turned positive, with a 1.2% increase in February 2025.  
*General Public*  
Costa Rica's economy has grown by 4.4% compared to last year, and unemployment is down to 9.7%. Inflation is also on the rise, with a 1.2% increase reported in February 2025.

---

**Mexico**  
*Expert*  
Mexico's IGAE shows a year-on-year decrease of approximately 0.2% for January-April 2025 compared to the same period in 2024, while unemployment remains stable at 2.6%. Month-over-month, inflation has slightly decreased to 3.8% in March 2025.  
*General Public*  
Mexico's economy has seen a slight decline of 0.2% compared to last year, and unemployment is steady at 2.6%. However, inflation has dipped a bit to 3.8% in March 2025, which is a positive sign for consumers.